In [ ]:
#regarder sessions dans request 

#selenium
#undetected 



# pour les catégories : regarder les chaussures (dockMartins)

#peut etre regarder concurrence (si plus de femmes vendent, fait baisser un peu les prix)
# regarder si vendeur est homme ou femme (avec profil ou autres articles vendus)

In [ ]:
import time
import requests
import pandas as pd
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import json

def fetch_vinted_items(search_text,
                       page=1,
                       per_page=96,
                       catalog_ids=257,
                       order='relevance',
                       size_ids='',
                       brand_ids='',
                       status_ids='',
                       color_ids='',
                       patterns_ids='',
                       material_ids=''):
    """
    Query Vinted internal API and return a pandas DataFrame of items (raw JSON normalized).
    Change `search_text` to perform different searches.
    """
    base_url = "https://www.vinted.fr/api/v2/catalog/items"
    params = {
        "page": page,
        "per_page": per_page,
        "time": int(time.time()),
        "search_text": search_text,
        "catalog_ids": catalog_ids,
        "order": order,
        "size_ids": size_ids,
        "brand_ids": brand_ids,
        "status_ids": status_ids,
        "color_ids": color_ids,
        "patterns_ids": patterns_ids,
        "material_ids": material_ids
    }

    s = requests.Session()
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
        "Accept": "application/json, text/plain, */*",
        "Referer": "https://www.vinted.fr/"
        # La valeur exacte trouvée dans l'onglet réseau sous 'csrf-token'
        "csrf-token": "ajax:123456789...", 
    
        # La longue chaîne de cookies copiée depuis le navigateur
        # Assurez-vous d'avoir 'li_at' et 'JSESSIONID' dedans
        "Cookie": 'li_at=AQEDAA...; JSESSIONID="ajax:123456789..."; ...'
    })

    resp = s.get(base_url, params=params, timeout=30)
    # resp.raise_for_status()
    data = resp.json()
    # save raw JSON to a file (derive .json from output_path if available)
    json_path = 'vinted_items.json'
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    items = data.get("items", [])
    # Normalize and return a DataFrame with raw JSON plus some common columns if present
    df = pd.json_normalize(items)
    if 'url' in df.columns:
        df['full_url'] = df['url'].apply(lambda u: f"https://www.vinted.fr{u}" if u and u.startswith('/') else u)
    return df

def parse_item_page(url):
    """
    Fetch an individual product page (HTML) and extract description + first image as example.
    Useful when API lacks some fields.
    """
    s = requests.Session()
    s.headers.update({"User-Agent": "Mozilla/5.0", "Referer": "https://www.vinted.fr/"})
    r = s.get(url, timeout=20)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    desc = soup.select_one("div[itemprop='description']") or soup.select_one(".description")
    images = soup.select("img")
    first_img = images[0]['src'] if images else None
    return {"description_html": str(desc) if desc else None, "first_image": first_img}

# Example usage:
df = fetch_vinted_items("Jeans", page=1, per_page=96)
df.head()

# details = parse_item_page(df.loc[0, "full_url"])  # if full_url exists

""


In [8]:
# export the existing dataframe `df` to CSV
output_path = "vinted_items.csv"
df.to_csv(output_path, index=False)
print(f"Saved {len(df)} rows to {output_path}")

Saved 0 rows to vinted_items.csv
